### Problem statement

The data is related to a coding platform that hosts coding challenges. They have a unique business model, where they crowdsource problems from various creators(authors). These authors create the problem and release it on the client's platform. The users then select the challenges they want to solve. 

This dataset contains information about each coding problem. It contains information about the problem, about the author who created it and a list of users who have attempted the problem.

Below are the fields that can be founf within each document in the collection -

- `challenge_id` - Unique id of the challenge problem

- `programming_language` - Programming language for the challenge

- `total_submissions` - Total submissions by all users

- `publish_date` - Publishing date for the challenge

- `author` - Embedded document about the author of the challenge.
> - `id` - Author id
> - `gender` - Author gender
> - `org_id` - Organisation if for author

- `users` - List of users who have attempted the challenge

----

### Connecting to MongoDB


----

In [1]:
# Importing the required libraries
import pymongo
import pprint as pp
from pymongo import MongoClient

pp.sorted = lambda x, key=None: x

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

---
### Importing data

----

In [ ]:
# # Restore database
#!mongorestore -d assignment C:\Users\chintamani_modak\Downloads\querying_assignment\querying_assignment\challenge.bson

In [3]:
db = client['exercise']

In [4]:
db.list_collection_names()

['challenge']

In [49]:
pp.pprint(
    db.challenge.find_one()
)

{'_id': ObjectId('60dab9f75945974466d8d64d'),
 'challenge_id': 'CI23478',
 'programming_language': 2,
 'total_submissions': 37,
 'publish_date': datetime.datetime(2006, 6, 5, 0, 0),
 'author': {'id': 'AI563576', 'gender': 'M', 'org_id': 'AOI100001'},
 'users': [32876, 88820, 97150, 97359]}


---
### Assignment Questions

----

### Q1. 

Find the number of documents in the collection

In [50]:
# Enter your code here
print(db.challenge.count_documents({}))

5606


### Q2. 

Find the number of unique `programming_language` and `challenge_id`

In [114]:
# Enter your code here
print(len(list(db.challenge.distinct("programming_language"))))
print(len(list(db.challenge.distinct("challenge_id"))))

3
5606


In [43]:
res = db.challenge.aggregate([{"$group":{"_id":"$programming_language","count":{"$count":{}}}}])
for i in res:
    print(i)

{'_id': 1, 'count': 5217}
{'_id': 2, 'count': 319}
{'_id': 3, 'count': 70}


### Q3. 

How many documents are there where the challenge was created between `2009-01-01` and `2010-01-01`? 

In [71]:
# Import datetime library
from datetime import datetime

In [108]:
# Enter your code here
print(db.challenge.count_documents({"publish_date":{"$gte":datetime(2009,1,1),"$lte":datetime(2010,1,1)}}))

888


### Q4. 

How many challenges have been written by author `AI563576` in either `programming_language` `1` or `3` ?


In [107]:
# Enter your code here
# Document where `limit` is either 1 or 3

db.challenge.count_documents({"$and":[{"author.id": "AI563576"},{"programming_language":{"$in":[1,3]}}]})                       

41

### Q5. 

How many documents are there where the challenges have been created by a female author and where the author belong to either the 'AOI100013' organisation or the 'AOI100013' organisation?

In [105]:
# Enter your code here
db.challenge.count_documents({"$and": [{"author.gender": "F"},{"$or":[{"author.org_id": {"$in": ["AOI100013", "AOI100014"]}}]}]})

5

### Q6.

Find the top 5 challenges where either the challenge has been attempted by exactly 100 `users` or where the `total_submissions` is between 100 and 200, both inclusive?

In [159]:
#Enter your code here
#print(db.challenge.find_one({"users":{"$eq":88820}}))
result = db.challenge.aggregate([
    {"$match":{
        "users":{"$ne":"None"}
    }},
    {"$project":{
        "_id":1,"challenge_id":1,"programming_language":1,'total_submissions':1,"publish_date":1,'total_number_of_users':{"$size":"$users"}
    }
    },
    {"$match":{"$or":[
        {"total_number_of_users":{"$gte":100},
        "$and":[{"total_submissions":{"$gte":100}},{"total_submissions":{"$lte":200}}]
        }
    ]}},
    {"$sort":{
        "total_submissions":-1,"total_number_of_users":-1
    }},
    {"$limit":5}
])
#result = db.challenge.aggregate([{"$project":{'array_length':{"$size":"$users"}}}])
#result = db.challenge.aggregate([{"$project":{"_id":0,'total_submissions':1,"users":1,'array_length':{"$size":"$users"}}},{"$sort":{"array_length":1}},{"$limit":5}])
for i in result:
    print(i)

{'_id': ObjectId('60dab9f85945974466d8e0a5'), 'challenge_id': 'CI26126', 'programming_language': 2, 'total_submissions': 200, 'publish_date': datetime.datetime(2008, 3, 29, 0, 0), 'total_number_of_users': 1003}
{'_id': ObjectId('60dab9f85945974466d8dbf5'), 'challenge_id': 'CI24926', 'programming_language': 1, 'total_submissions': 200, 'publish_date': datetime.datetime(2007, 3, 7, 0, 0), 'total_number_of_users': 374}
{'_id': ObjectId('60dab9f75945974466d8d75e'), 'challenge_id': 'CI23751', 'programming_language': 1, 'total_submissions': 200, 'publish_date': datetime.datetime(2006, 4, 10, 0, 0), 'total_number_of_users': 115}
{'_id': ObjectId('60dab9f85945974466d8e0aa'), 'challenge_id': 'CI26131', 'programming_language': 1, 'total_submissions': 199, 'publish_date': datetime.datetime(2008, 3, 26, 0, 0), 'total_number_of_users': 2820}
{'_id': ObjectId('60dab9f95945974466d8e614'), 'challenge_id': 'CI27535', 'programming_language': 1, 'total_submissions': 198, 'publish_date': datetime.datetime

### Q7. 

How documents are there where either the `publish_date > 2010-01-01` and `total_submissions > 100`, or the `publish_date < 2000-01-01` and `total_submissions > 1000` ?

In [72]:
# Enter your code here
print(db.challenge.count_documents({"$and": [{"publish_date":{"$gt":datetime(2010,1,1)}},{"total_submissions":{"$gt":100}}]}))

12


In [73]:
print(db.challenge.count_documents({"$and": [{"publish_date":{"$lt":datetime(2000,1,1)}},{"total_submissions":{"$gt":1000}}]}))

33


In [104]:
print(db.challenge.count_documents({"$or":[{"$and": [{"publish_date":{"$gt":datetime(2010,1,1)}},{"total_submissions":{"$gt":100}}]},{"$and": [{"publish_date":{"$lt":datetime(2000,1,1)}},{"total_submissions":{"$gt":1000}}]}]}))

45
